# Creación de Pandas DataFrames a partir de peticiones API
En este ejemplo, utilizaremos la API del U.S. Geological Survey para obtener un objeto JSON de datos de terremotos y convertirlo en un `pandas.DataFrame`.

API DEL USGS: https://earthquake.usgs.gov/fdsnws/event/1/

### Obtener datos de la API

In [1]:
import datetime as dt   # para pedir todas las fechas que quiera
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# asegurémonos de que la solicitud fue correcta
response.status_code

200

Una respuesta de 200 significa OK, así que podemos extraer los datos del resultado. Como le pedimos a la API una carga JSON, podemos extraerla de la respuesta con el método `json()`.

### Aislar los datos de la respuesta JSON
Necesitamos comprobar las estructuras de los datos de la respuesta para saber dónde están nuestros datos.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

La API del USGS proporciona información sobre nuestra solicitud en la clave `metadata`. Tenga en cuenta que el resultado será diferente, independientemente del intervalo de fechas que elija, ya que la API incluye una marca de tiempo que indica cuándo se extrajeron los datos:

In [3]:
earthquake_json['metadata']

{'generated': 1701083577000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-10-27&endtime=2023-11-26',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 9621}

Cada elemento de la matriz JSON `features` es una fila de datos para nuestro marco de datos.

In [4]:
type(earthquake_json['features'])

list

Sus datos serán diferentes dependiendo de la fecha en que ejecute esto.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.79999995,
  'place': '13 km SSW of Volcano, Hawaii',
  'time': 1700955832290,
  'updated': 1700956011470,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv73663117',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv73663117&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 50,
  'net': 'hv',
  'code': '73663117',
  'ids': ',hv73663117,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 17,
  'dmin': None,
  'rms': 0.0900000036,
  'gap': 71,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 1.8 - 13 km SSW of Volcano, Hawaii'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.299499511719, 19.3395004272461, 2.1800000667572]},
 'id': 'hv73663117'}

In [6]:
earthquake_json['features'][0].keys()

dict_keys(['type', 'properties', 'geometry', 'id'])

### Convertir a DataFrame
Necesitamos tomar la sección `properties` de cada entrada del array JSON `features` para crear nuestro dataframe.

In [8]:
pd.DataFrame(earthquake_json['features'])

,type,properties,geometry,id
0,Feature,"{'mag': 1.79999995, 'place': '13 km SSW of Vol...","{'type': 'Point', 'coordinates': [-155.2994995...",hv73663117
1,Feature,"{'mag': 1.95, 'place': '1 km N of Prattville, ...","{'type': 'Point', 'coordinates': [-121.1578369...",nc73967466
2,Feature,"{'mag': 0.74, 'place': '14 km SSW of Searles V...","{'type': 'Point', 'coordinates': [-117.4528333...",ci39717962
3,Feature,"{'mag': 0.61, 'place': None, 'time': 170095471...","{'type': 'Point', 'coordinates': [-155.2801666...",hv73663092
4,Feature,"{'mag': 0.66, 'place': 'Island of Hawaii, Hawa...","{'type': 'Point', 'coordinates': [-155.2531666...",hv73663072
...,...,...,...,...
9616,Feature,"{'mag': 0.3, 'place': '8 km NW of The Geysers,...","{'type': 'Point', 'coordinates': [-122.8188333...",nc73951885
9617,Feature,"{'mag': 4.1, 'place': '283 km SSW of Severo-Ku...","{'type': 'Point', 'coordinates': [154.8447, 48...",us7000l82p
9618,Feature,"{'mag': 1.7, 'place': '6 km NNE of Pecos, Texa...","{'type': 'Point', 'coordinates': [-103.476, 31...",tx2023vahp
9619,Feature,"{'mag': 2.6, 'place': '111 km S of False Pass,...","{'type': 'Point', 'coordinates': [-163.1249, 5...",us6000lnn7


In [7]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
# lo que hago es iterar por cada elemento de la lista "earthquake_json['features']" y cuando el elemento sea "quake['properties']" lo anexo
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.80,"13 km SSW of Volcano, Hawaii",1700955832290,1700956011470,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73663117,",",hv,",",origin,phase-data,",17.0,NaN,0.09,71.0,md,earthquake,"M 1.8 - 13 km SSW of Volcano, Hawaii"
1,1.95,"1 km N of Prattville, CA",1700955302300,1700967733993,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73967466,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",5.0,0.2418,0.01,193.0,md,earthquake,"M 2.0 - 1 km N of Prattville, CA"
2,0.74,"14 km SSW of Searles Valley, CA",1700955266830,1701015584009,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39717962,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",22.0,0.0275,0.13,59.0,ml,earthquake,"M 0.7 - 14 km SSW of Searles Valley, CA"
3,0.61,None,1700954715210,1700956082230,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73663092,",",hv,",",origin,phase-data,",18.0,NaN,0.07,58.0,md,earthquake,M 0.6 -
4,0.66,"Island of Hawaii, Hawaii",1700954346350,1700955688640,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73663072,",",hv,",",origin,phase-data,",16.0,NaN,0.08,86.0,md,earthquake,"M 0.7 - Island of Hawaii, Hawaii"


### (Opcional) Guardar los datos en un CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creando_dataframes.ipynb">
        <button style="float: left;">&#8592; Notebook Anterior</button>
    </a>
    <a href="./4-inspeccionando_dataframes.ipynb">
        <button style="float: right;">Siguiente Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>